# Robust Portfolio

In [1]:
import cvxpy as cp
import numpy as np
import torch
import matplotlib.pyplot as plt

import lropt
from lropt.parameter import Parameter
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

In [16]:
n = 150
i = np.arange(1, n+1)
p = 1.15 + i*0.05/150
delta = np.array(0.05/450 * (2*i*n*(n+1))**0.5)
Gamma = 5

In [17]:
x = cp.Variable(n)
t = cp.Variable()
uncertainty_set = lropt.Budget(rho1=1, rho2=Gamma)
z = lropt.UncertainParameter(n, uncertainty_set=uncertainty_set)
objective = cp.Maximize((p + cp.multiply(delta,z)) @ x)
constraints = [cp.sum(x)==1,
                x>=0,
              ]
# objective = cp.Minimize(t)
# constraints = [-(p + cp.multiply(delta,z)) @ x <= t,
#                 cp.sum(x)==1,
#                 x>=0,
#               ]
prob = RobustProblem(objective=objective, constraints=constraints)

In [18]:
prob.solve(solver="CLARABEL")

1.184443027116401